FOMO 公式

1. FOMO 前 (D-1 day)：條件日成立的 FOMO day 的前一日

2. FOMO 中 (D-day)：
    a) 歷史交易日中，成交量(volume) 最高的前 3 個日期
    b) 第一高 減去 第二高、第三高的日期，其成交量比例應小於 20 % 才選入 FOMO 日 (有可能部份項目 FOMO 日少於三個日期)
    
3. FOMO 後 (D+1 day)：條件日成立的 FOMO day 的後一日


In [ ]:
import json
import numpy as np
import pandas as pd
import datetime
import os
import time

#'NFT20_success改版.feather' 同 "NFT20_successful_events_new_有winneraddress.xlsx"
data_dir = os.path.join(os.getcwd(), 'data')
df = os.path.join(data_dir, 'NFT20_success改版.feather')

start_time = time.time()
df_all = pd.read_feather(df)
total_time = time.time() - start_time
print("Total seconds to load:", total_time)

In [ ]:
#21個項目交易單量按日加總
df_all["event_timestamp"] = pd.to_datetime(df_all['event_timestamp'].copy())
grp = df_all.query('deal_price > 0').groupby(pd.Grouper(key='event_timestamp', freq='1D'))
grp = grp.size()
grp.name = "count_all"
grp = pd.DataFrame(grp)


In [ ]:
#套入Fomo公式
grp = grp.sort_values(by="count_all", ascending=False)
grp['pct'] = 100 * (1 - grp["count_all"] / grp.iloc[0,0])
grp['pct_within20'] =  [grp['pct']<20][0]

In [ ]:
#項目清單
g = df_all.groupby("collection_slug")
groups_collection = list(g.groups.keys())

#個別項目套用公式
df_collections = pd.DataFrame()
for i in groups_collection:
    grp_temp = df_all.query('collection_slug == "' + i + '"& deal_price > 0').groupby(pd.Grouper(key='event_timestamp', freq='1D'))
    grp2 = grp_temp.size()
    grp2.name = "count_collections"
    grp2 = pd.DataFrame(grp2)
    grp2 = grp2.sort_values(by="count_collections", ascending=False)
    grp2['pct'] = 100 * (1 - grp2["count_collections"] / grp2.iloc[0,0])
    grp2['pct_within20'] =  [grp2['pct']<20][0]
    grp2 = grp2[grp2['pct_within20']==True]
    grp2["collection_slug"] = i
    df_collections = df_collections.append(grp2)

df_collections2 = pd.concat([df_collections, grp["count_all"]], axis=1).reindex(df_collections.index)
df_collections2["交易單量佔總數"] = df_collections2["count_collections"] / df_collections2["count_all"]


In [ ]:
#Fomo day=1、前一日=-1、後一日=2、其他=0
fomodf = pd.DataFrame([1]*len(df_collections2.index),index=df_collections2.index,columns=["Fomoday"])
grp = pd.concat([grp, fomodf.reindex(grp.index)], axis=1)
grp = grp.sort_values(by="event_timestamp", ascending=True)
grp["Fomoday_p"] = grp["Fomoday"].fillna(limit=1,method="backfill")
grp["Fomoday_last"] = grp["Fomoday_p"].fillna(limit=1,method='ffill')
grp["Fomoday"]= np.where(grp["Fomoday_p"]==1 & pd.isna(grp["Fomoday"]), -1 ,grp["Fomoday"])
grp["Fomoday"]= np.where(grp["Fomoday_last"]==1 & pd.isna(grp["Fomoday"]), 2 ,grp["Fomoday"])
grp["Fomoday"] = grp["Fomoday"].fillna(0)

grp["datetime"] = grp.index.to_pydatetime()
grp["datetime"] = grp["datetime"].dt.date


In [ ]:
#倂回原始資料
df_all["datetime"] = df_all["event_timestamp"].dt.date
result = pd.merge(df_all, grp[["datetime","Fomoday"]], on="datetime")